# 06. Modular RAG - 기능 맛보기!

## 🎯 학습 목표
- **Query Routing**: LLM이 질문 유형을 판별하여 자동 분류
- **Self-RAG Critic**: LLM이 자신의 답변을 평가하고 개선점 찾기
- **Retry with Refine**: 부족한 부분을 보강해서 재시도하는 지능형 시스템

## 📋 실습 구성
1. **Query Routing 맛보기** (10분) - 질문을 single/multi/clarify로 분류
2. **Self-RAG Critic 맛보기** (10분) - 답변을 스스로 평가하고 개선점 찾기  
3. **Retry with Refine 맛보기** (10분) - 누락된 부분 보강해서 재시도

---

> 💡 **핵심 아이디어**: LLM이 스스로 분류→평가→보강하는 **메타인지 능력**을 체험해보세요! 05번 환경을 그대로 사용하면서 간단한 기능들을 독립적으로 맛봅니다.

In [ ]:
# 05번 환경 재사용 (Day1FinetunedLLM + 항공 코퍼스)
!pip install -q langchain-community faiss-cpu sentence-transformers matplotlib pandas numpy transformers torch rank-bm25

import os
import time
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple
import json
import re
import warnings
warnings.filterwarnings('ignore')

# 🤖 Day1 파인튜닝 모델 관련 import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms.base import LLM
from pydantic import Field

# LangChain 관련 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

print("✅ 라이브러리 설정 완료!")

# Day 1 파인튜닝 모델 클래스 (05번과 동일)
class Day1FinetunedLLM(LLM):
    """Day 1 파인튜닝 모델을 사용하는 LLM 클래스"""
    
    # Pydantic 필드 선언
    model_name: str = Field(default="ryanu/my-exaone-raft-model")
    tokenizer: Any = Field(default=None)
    model: Any = Field(default=None)
    
    class Config:
        arbitrary_types_allowed = True
    
    def __init__(self, model_name: str = "ryanu/my-exaone-raft-model", **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        print(f"🎯 Day 1 파인튜닝 모델 로드: {self.model_name}")
        self._load_model()
    
    def _load_model(self):
        """모델 로드"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        print("✅ 모델 로드 완료!")
    
    @property
    def _llm_type(self) -> str:
        return "day1_finetuned_llm"
    
    def _call(self, prompt: str, stop=None, run_manager=None, **kwargs) -> str:
        """실제 모델 추론"""
        # EXAONE 프롬프트 템플릿 적용
        formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
        
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs, max_new_tokens=512, temperature=0.7, do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id, eos_token_id=self.tokenizer.eos_token_id
            )
        
        response = self.tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        return response if response else "적절한 답변을 생성할 수 없습니다."

# LLM 초기화
llm = Day1FinetunedLLM()

print("✅ Day1FinetunedLLM 초기화 완료!")

In [ ]:
# 05번 항공 코퍼스 재사용 + 벡터스토어 구축
def create_aviation_corpus():
    """05번과 동일한 항공사 + 공항 코퍼스"""
    return [
        {
            "content": """에어민트 항공 수하물 정책 (2024-05-10)
- 위탁수하물: 15kg 무료, 초과 1kg당 8,000원
- 기내수하물: 7kg 1개
- 노선: 국내선 동일 기준""",
            "metadata": {"type": "airline_policy", "airline": "에어민트", "source": "official", "updated_at": "2024-05-10"}
        },
        {
            "content": """김포공항 보안검색 혼잡도 안내 (2024-04-15)
- 일요일: 08~10시 혼잡, 12~14시는 비교적 한산
- 평일: 출근시간대 07~09시 혼잡
- 오후 시간대는 전반적으로 여유로움""",
            "metadata": {"type": "airport_info", "airport": "김포", "source": "official", "updated_at": "2024-04-15"}
        },
        {
            "content": """인천공항 보안검색 혼잡도 (2024-04-20)
- 주말: 09~11시 피크, 15~17시 한산
- 국제선 터미널은 별도 시간대 적용""",
            "metadata": {"type": "airport_info", "airport": "인천", "source": "official", "updated_at": "2024-04-20"}
        },
        {
            "content": """스카이블루 항공 수하물 정책 (2024-03-01)
- 위탁수하물: 20kg 무료
- 기내수하물: 10kg 1개""",
            "metadata": {"type": "airline_policy", "airline": "스카이블루", "source": "official", "updated_at": "2024-03-01"}
        },
        {
            "content": """공항 빠르게 통과하는 여행 팁
- 수하물 가볍게 준비하기
- 이른 시간 도착 추천
- 주말 오전 피크타임은 피하기""",
            "metadata": {"type": "blog", "source": "blog", "updated_at": "2024-08-01"}
        }
    ]

# 벡터스토어 구축
aviation_docs = create_aviation_corpus()
lc_documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in aviation_docs]

# 임베딩 + 벡터스토어 (05번과 동일)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(lc_documents, embeddings)

# 간단한 RAG 체인용 리트리버
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f"✅ 항공 코퍼스 벡터스토어 구축 완료! ({len(aviation_docs)}개 문서)")
print("\n📋 문서 목록:")
for i, doc in enumerate(aviation_docs, 1):
    doc_type = doc["metadata"].get("type", "unknown")
    entity = doc["metadata"].get("airline") or doc["metadata"].get("airport", "일반")
    first_line = doc["content"].strip().split('\n')[0]
    print(f"  {i}. [{doc_type}|{entity}] {first_line}")

In [ ]:
# 🎯 Step 1: Query Routing 맛보기 - 하드코딩 vs LLM

# 📋 1. 하드코딩 라우팅 (규칙 기반)
def hardcoded_routing(query):
    """키워드 기반 간단한 라우팅"""
    query_lower = query.lower()
    
    if "수하물" in query_lower or "짐" in query_lower:
        return "baggage_fee_search", "수하물 요금 검색 함수"
    elif "보안검색" in query_lower or "검색대" in query_lower:
        return "security_time_search", "보안검색 시간 검색 함수"  
    elif "계산" in query_lower or "더하기" in query_lower or "빼기" in query_lower:
        return "calculator", "계산기 함수"
    elif "날씨" in query_lower:
        return "weather_search", "날씨 검색 함수"
    else:
        return "general_rag_search", "일반 RAG 검색 함수"

# 🤖 2. LLM 라우팅 (의미 기반)
def llm_routing(llm, query):
    """LLM이 어떤 함수로 보낼지 결정"""
    
    prompt = f"""다음 질문을 보고 어떤 함수로 보내야 할지 결정하세요.

가능한 함수들:
- baggage_fee_search: 수하물 요금 관련
- security_time_search: 공항 보안검색 시간 관련  
- calculator: 계산 관련
- weather_search: 날씨 관련
- general_rag_search: 일반적인 정보 검색

질문: {query}

답변 형식: 함수명|이유
예: baggage_fee_search|수하물 요금을 묻고 있음"""

    response = llm._call(prompt)
    
    # 간단하게 파싱 (|로 분리)
    if "|" in response:
        func_name, reason = response.split("|", 1)
        return func_name.strip(), reason.strip()
    else:
        return "general_rag_search", "LLM 파싱 실패"

# 🔎 하드코딩 vs LLM 라우팅 비교 테스트
print("🎯 Query Routing 맛보기: 하드코딩 vs LLM")
print("="*60)

test_queries = [
    "에어민트 수하물 요금 알려줘",
    "김포공항 보안검색 언제가 빨라?", 
    "2+3은 얼마야?",
    "내일 서울 날씨 어때?",
    "항공사 정책에 대해 알고 싶어"
]

print("💡 같은 질문, 다른 라우팅 방식!")
print()

for i, query in enumerate(test_queries, 1):
    print(f"{i}. 질문: '{query}'")
    
    # 하드코딩 라우팅
    hard_func, hard_reason = hardcoded_routing(query)
    print(f"   🔧 하드코딩: {hard_func} ({hard_reason})")
    
    # LLM 라우팅  
    llm_func, llm_reason = llm_routing(llm, query)
    print(f"   🤖 LLM: {llm_func} ({llm_reason})")
    
    # 결과 비교
    if hard_func == llm_func:
        print("   ✅ 결과 일치!")
    else:
        print("   🔄 결과 다름 (LLM이 더 유연한 이해)")
    print()

print("🚀 하드코딩 vs LLM 라우팅 비교:")
print("• 하드코딩: 빠름, 명확함, 하지만 제한적")
print("• LLM: 느림, 유연함, 복잡한 상황 대응")
print("• 실무: 명확한 케이스는 하드코딩, 애매한 케이스는 LLM!")

print("\n💡 핵심: '라우팅 = 적절한 함수 선택'")
print("• 수하물 문의 → baggage_fee_search() 호출")
print("• 계산 요청 → calculator() 호출")
print("• 목적지 함수만 정확히 찾으면 성공!")

In [ ]:
# 🎯 Step 2: Self-RAG Critic 맛보기

# (A) 기본 RAG로 답변 생성 (Step 1 라우팅 결과 활용 가능)
STRICT_PROMPT = PromptTemplate.from_template(
"""아래 컨텍스트에 있는 내용만 근거로 한국어로 간결히 답하세요.
없으면 '근거 불충분: (무엇이 필요한지)' 한 줄로 말하고, 필요한 추가정보 1~2개만 물어보세요.

[컨텍스트]
{context}

[질문]
{question}

[답변]""")

def answer_with_context(llm, docs, question):
    """컨텍스트 기반 RAG 답변"""
    context = "\n\n---\n\n".join([d.page_content[:1000] for d in docs]) or "(빈 컨텍스트)"
    prompt_text = STRICT_PROMPT.format(context=context, question=question)
    response = llm._call(prompt_text)
    return response.strip()

# (B) Self-RAG Critic: LLM이 답변을 자가평가
CRITIC_PROMPT = PromptTemplate.from_template(
"""당신은 품질 심사관입니다. 아래를 보고 JSON만 출력하세요.

- coverage: 0.0~1.0 (질문 요구 파셋을 얼마나 근거로 커버?)
- missing_facets: 누락된 축 목록
- hallucination_risk: "low"|"medium"|"high"
- ask: 사용자에게 물어볼 1~2개 질문

[질문]
{question}

[답변]
{answer}

[컨텍스트 요약]
{ctx}

JSON:""")

def self_rag_critic(llm, question, answer, docs):
    """LLM이 Self-RAG로 답변 품질 평가"""
    ctx = "\n---\n".join([d.page_content[:400] for d in docs[:3]])
    prompt_text = CRITIC_PROMPT.format(question=question, answer=answer, ctx=ctx)
    response = llm._call(prompt_text)
    return _safe_json(response, {
        "coverage": 0.0, 
        "missing_facets": [], 
        "hallucination_risk": "medium", 
        "ask": []
    })

# 🔎 Self-RAG Critic 테스트  
print("🧠 Self-RAG Critic 맛보기")
print("="*50)

test_query = "김포공항에서 에어민트 타고 수하물 요금이랑 보안검색 시간도 알려줘"
print(f"질문: '{test_query}'\n")

# Step 1에서 라우팅한 결과도 활용 가능
routing_result = llm_route_query(llm, test_query)
print(f"📍 Step 1 라우팅 결과: {routing_result['route']} (핵심축: {routing_result['facets']})")

# 1차: 기본 RAG 답변
docs = retriever.get_relevant_documents(test_query)
print(f"\n🔍 검색된 문서:")
for i, doc in enumerate(docs, 1):
    doc_type = doc.metadata.get("type", "unknown")
    entity = doc.metadata.get("airline") or doc.metadata.get("airport", "-")
    print(f"  {i}. [{doc_type}|{entity}] {doc.page_content[:50]}...")

answer = answer_with_context(llm, docs, test_query)
print(f"\n💬 RAG 답변:\n{answer}")

# 2차: Self-RAG 평가
critic_result = self_rag_critic(llm, test_query, answer, docs)

coverage = critic_result.get("coverage", 0.0)
missing_facets = critic_result.get("missing_facets", [])
hallucination_risk = critic_result.get("hallucination_risk", "medium")
ask_questions = critic_result.get("ask", [])

print(f"\n🤔 LLM Self-RAG 평가:")
print(f"   커버리지: {coverage:.1f} (0.0~1.0)")
print(f"   누락된 축: {missing_facets}")
print(f"   할루시네이션 위험: {hallucination_risk}")
print(f"   추가 질문: {ask_questions}")

# 품질 해석
if coverage >= 0.8 and hallucination_risk == "low":
    quality = "✅ 우수"
elif coverage >= 0.6:
    quality = "⚠️ 보통"
else:
    quality = "❌ 개선 필요"

print(f"   종합 평가: {quality}")

print(f"\n💡 Self-RAG의 메타인지 능력:")
print(f"• LLM이 스스로 답변의 완성도를 객관 평가")
print(f"• 누락 파셋을 정확히 감지 (Step 1 라우팅과 연계)") 
print(f"• 할루시네이션 위험도를 사전 경고")

In [ ]:
# 🎯 Step 3: Retry with Refine 맛보기

# 누락된 파셋으로 정밀 쿼리 생성
REFINE_PROMPT = PromptTemplate.from_template(
"""아래 질문의 누락된 파셋을 메우는 '정밀 검색쿼리' 2개만 JSON 배열로 출력하세요.
- 각 쿼리는 파셋을 명시적으로 포함(브랜드/도시/요일/정책명 등)
- 중복 금지
- 쿼리로 유추할 수 있는 필요한 내용만 적으세요

[질문]
{question}

[누락 파셋]
{missing_facets}

JSON:""")

def refine_queries(llm, question, missing_facets):
    """누락 파셋으로 보강 쿼리 생성"""
    if not missing_facets: 
        return []
    prompt_text = REFINE_PROMPT.format(question=question, missing_facets=missing_facets)
    response = llm._call(prompt_text)
    arr = _safe_json(response, [])
    return [q for q in arr if isinstance(q, str)][:2]

def retry_once_with_refine(llm, retriever, question):
    """1회 재시도로 답변 품질 개선"""
    
    # 1차 시도
    print("🔵 1차 시도:")
    docs = retriever.get_relevant_documents(question)
    answer = answer_with_context(llm, docs, question)
    critic_result = self_rag_critic(llm, question, answer, docs)
    
    print(f"   답변: {answer}")
    print(f"   평가: 커버리지 {critic_result['coverage']:.1f}, 위험도 {critic_result['hallucination_risk']}")
    
    # 품질이 충분하면 종료
    if critic_result["coverage"] >= 0.8 and critic_result["hallucination_risk"] == "low":
        print("   ✅ 품질 충분! 1차 답변으로 완료")
        return answer
    
    # 보강이 필요하면 재시도
    print("   ⚠️ 품질 개선 필요, 재시도 진행...")
    
    # 보강 쿼리 생성
    refine_queries_list = refine_queries(llm, question, critic_result["missing_facets"])
    if not refine_queries_list:
        return f"근거 불충분. 추가정보 필요: {' / '.join(critic_result.get('ask', []))}"
    
    print(f"\n🟢 2차 시도 (보강 쿼리: {refine_queries_list}):")
    
    # 보강 문서 수집
    additional_docs = []
    for refine_query in refine_queries_list:
        additional_docs += retriever.get_relevant_documents(refine_query)[:2]
    
    # 중복 제거 (간단하게 content 기준)
    seen_content = set()
    unique_docs = []
    for doc in additional_docs:
        content_key = doc.page_content[:100]  # 앞 100자로 중복 판단
        if content_key not in seen_content:
            seen_content.add(content_key)
            unique_docs.append(doc)
    
    # 2차 답변 생성
    final_answer = answer_with_context(llm, unique_docs[:6], question)
    final_critic = self_rag_critic(llm, question, final_answer, unique_docs[:6])
    
    print(f"   답변: {final_answer}")
    print(f"   평가: 커버리지 {final_critic['coverage']:.1f}, 위험도 {final_critic['hallucination_risk']}")
    
    # 개선 효과 표시
    improvement = final_critic["coverage"] - critic_result["coverage"]
    print(f"   📈 개선도: {improvement:+.1f} 포인트")
    
    return final_answer

# 🔎 Retry with Refine 전체 테스트
print("🔄 Retry with Refine 맛보기")
print("="*50)

complex_query = "김포공항에서 에어민트 타고 수하물 요금이랑 보안검색 시간도 알려줘"
print(f"복합 질문: '{complex_query}'\n")

final_answer = retry_once_with_refine(llm, retriever, complex_query)

print(f"\n✅ 최종 결과:\n{final_answer}")

print(f"\n💡 Retry with Refine 핵심:")
print(f"• 1차 답변을 Self-RAG가 품질 평가")
print(f"• 누락된 파셋을 LLM이 자동 감지") 
print(f"• 정밀 검색쿼리로 보강 정보 수집")
print(f"• 2차 답변으로 품질 향상 달성")

## 🎯 Modular RAG 기능 맛보기 완료!

### 📊 3단계 기능 체험 성과
- **🎯 Query Routing**: LLM이 질문을 single/multi/clarify로 자동 분류
- **🧠 Self-RAG Critic**: LLM이 자신의 답변을 객관적으로 평가 (커버리지/할루시네이션/누락 파셋)
- **🔄 Retry with Refine**: 부족한 부분을 정밀 검색으로 보강해서 재시도

### 💡 핵심 인사이트
1. **메타인지 능력**: LLM이 분류→평가→보강의 고차원적 사고 수행
2. **자가 개선**: 스스로 부족함을 감지하고 개선하는 능력
3. **모듈화**: 각 기능이 독립적이면서도 유기적으로 연결

### 🛠 실무 적용 팁
- **Query Routing**: coverage 기준을 0.8→0.6으로 낮춰 민감도 조절
- **Self-RAG**: 할루시네이션 임계치로 답변 신뢰성 보장
- **Retry**: k=3→k=5로 늘려 보강 문서 수 조절

### 🚀 다음 단계 확장
- 더 정교한 라우팅 전략 (BM25 vs Vector vs Hybrid 선택)
- Multi-hop Self-RAG (여러 번 반복 개선)
- 성능 기반 Adaptive Selection (잘 되는 전략 학습)

---

🎉 **06번 모듈러 RAG 맛보기 완료!** 이제 LLM의 메타인지 능력을 활용한 지능형 RAG 시스템을 경험하셨습니다!